### Deadline: 12:00 08.11 (Tuesday) 
**Group size: 2-3 persons.** <br>
**Assignments should be performed in this ipython notebook, saved and sent to vasilev@uni-koblenz.de, florian.lemmerich@gesis.org, philipp.singer@gesis.org with Subject: [ML-Assignment]**<br>
**You create groups on your own, names of all group participants should be mentioned in the letter.** 

###  Following cell is for data preparation, you should put 'adult.data' file in the same folder as current notebook and ran the cell in the beginning.

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
names = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain',
        'capital-loss','hours-per-week','native-country','income']
df = pd.read_csv('adult.data',names=names,index_col=False)
df = df[['age','workclass','sex','hours-per-week','education','capital-gain','capital-loss','income']]
df.replace(' ?',np.nan,inplace=True)

### The dataset consists of 7 features of a person and an income class that they belong to: '>50K' or '<=50K'.

In [8]:
df.head()

,age,workclass,sex,hours-per-week,education,capital-gain,capital-loss,income
0,39,State-gov,Male,40,Bachelors,2174,0,<=50K
1,50,Self-emp-not-inc,Male,13,Bachelors,0,0,<=50K
2,38,Private,Male,40,HS-grad,0,0,<=50K
3,53,Private,Male,40,11th,0,0,<=50K
4,28,Private,Female,40,Bachelors,0,0,<=50K


## 1) Perform k-nearest neighbors algorithm with two k parameters of your choice on a given dataset.

### 1.1) Preprocessing: dataset contains missing values and categorical variables, you need to handle them, before applying an algorithm on the data.

In [9]:
df = df.dropna()#removing irrelevant row from dataset i.e. np.nan

#Each string value per column is replaced with integer value
df_columnNames = ['workclass' , 'sex' ,'education'] 
uniqueValueColumns = [set(df[i]) for i in df_columnNames]
dictIntegerValueAdded = [dict(zip(uniqueValuePerColumn , [k for k in range(0,len(uniqueValuePerColumn))] )) for uniqueValuePerColumn in uniqueValueColumns]

#Displaying the dictionary : mapped string value per column to integer
print (dictIntegerValueAdded[0])
print (dictIntegerValueAdded[1])
print (dictIntegerValueAdded[2])

#Replace the dataframe string values with above created integer value i.e from dictionary 
for i,columnName in enumerate(df_columnNames):
    df = df.replace({columnName: dictIntegerValueAdded[i]}) 

{' Self-emp-not-inc': 5, ' Self-emp-inc': 7, ' State-gov': 0, ' Federal-gov': 1, ' Without-pay': 3, ' Local-gov': 4, ' Private': 6, ' Never-worked': 2}
{' Male': 0, ' Female': 1}
{' Assoc-acdm': 0, ' 7th-8th': 1, ' HS-grad': 6, ' Prof-school': 3, ' Doctorate': 5, ' 1st-4th': 7, ' 9th': 8, ' Preschool': 9, ' Bachelors': 10, ' 5th-6th': 11, ' Some-college': 15, ' 11th': 12, ' 10th': 4, ' 12th': 13, ' Masters': 14, ' Assoc-voc': 2}


### 1.2) You need to divide your dataset into two parts: training and test. Training subset should contain 80% of the whole dataset and target classes should be balanced in both subsets.

In [10]:
#For balancing the subsets - target classes
AllLessEqualFiftyK = df[df['income']==' <=50K']  #i.e we take 80% 20% from this dataset (<=50K)
AllGreaterFiftyK = df[df['income']==' >50K']    # i.e we take 80% 20% from this dataset (>50K)

# Generate 80% : 20% of training and test set for <=50K
trainLessEqualFiftyK = AllLessEqualFiftyK.sample(frac=0.8, random_state=1)
# Select anything not in the training set and put it in the testing set.
testLessEqualFiftyK = AllLessEqualFiftyK.loc[~AllLessEqualFiftyK.index.isin(trainLessEqualFiftyK.index)]


#Generate 80% : 20% of training and test set for >50K
trainGreaterFiftyK = AllGreaterFiftyK.sample(frac=0.8, random_state=1)
# Select anything not in the training set and put it in the testing set.
testGreaterFiftyK = AllGreaterFiftyK.loc[~AllGreaterFiftyK.index.isin(trainGreaterFiftyK.index)]

#Merging the balanced dataset to obtain train and test data 
train = trainLessEqualFiftyK.append(trainGreaterFiftyK)
test = testLessEqualFiftyK.append(testGreaterFiftyK)


### 1.3) Apply k-nearest neighbors algorithm with two different k parameters of your choice:

In [11]:
from sklearn.neighbors import KNeighborsClassifier #Importing the classifier from scikit learn

X_train = train[['age','workclass','sex','hours-per-week','education','capital-gain','capital-loss']] # Data that we will be training on
y_train = train['income'] # Labels of training data
X_test = test[['age','workclass','sex','hours-per-week','education','capital-gain','capital-loss']] # Data points which we will be testing


knn1 = KNeighborsClassifier(n_neighbors=1) # Initializing first classifier with number of neighbours = 1
knn9 = KNeighborsClassifier(n_neighbors=9) # Initializing second classifier with number of neighbours = 9

knn1.fit(X_train,y_train);
knn9.fit(X_train,y_train);

# Prediction : k nearest neighbors algoritm with k = 1
result1 = knn1.predict(X_test) #Predicting class of new customers with 
# Prediction : k nearest neighbors algoritm with k = 9
result9 = knn9.predict(X_test)
print("With k=1 the new customers will have following classes:",result1)
print("With k=9 the new customers will have following classes:",result9)



With k=1 the new customers will have following classes: [' <=50K' ' <=50K' ' <=50K' ..., ' >50K' ' >50K' ' <=50K']
With k=9 the new customers will have following classes: [' <=50K' ' <=50K' ' <=50K' ..., ' >50K' ' >50K' ' <=50K']


## 2) Evaluate and compare performance of two models:

### 2.1) Print performance metrics of your models:

In [12]:
#importing report module
from sklearn.metrics import classification_report
rep1 = classification_report(test['income'],result1) # Module takes in list of correct classes and list of predicted classes
rep9 = classification_report(test['income'],result9)
print ("FOR K = 1")
print(rep1)
print("------------------------------------------------------")
print ("FOR K = 9")
print(rep9)
print("------------------------------------------------------")

print ("TOTAL CORRECT")
print ("for K = 1")
print("{0} / {1} correct".format(np.sum(test['income'] == result1), len(test['income'])))

print ("for K = 9")
print("{0} / {1} correct".format(np.sum(test['income'] == result9), len(test['income'])))

FOR K = 1
             precision    recall  f1-score   support

      <=50K       0.84      0.94      0.89      4615
       >50K       0.72      0.44      0.55      1530

avg / total       0.81      0.82      0.80      6145

------------------------------------------------------
FOR K = 9
             precision    recall  f1-score   support

      <=50K       0.84      0.96      0.89      4615
       >50K       0.77      0.43      0.55      1530

avg / total       0.82      0.83      0.81      6145

------------------------------------------------------
TOTAL CORRECT
for K = 1
5024 / 6145 correct
for K = 9
5081 / 6145 correct


### 2.2) In a few sentences argue which k performed better, based on performance metrics from the previous task. 

### 2.3) Classification for which class were performed better? Why do you think this is the case?

In [13]:
print ("Total <=50K Training" , train[train['income']==' <=50K'].shape)
print ("Total >50K Training" , train[train['income']==' >50K'].shape)

Total <=50K Training (18460, 8)
Total >50K Training (6120, 8)
